# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie domowe 2

In [1]:
import pandas as pd
import numpy as np

## Zadanie 1 - eksploracja przestrzeni zagnieżdżeń
Wczytajmy do przestrzeni plik zagnieżdżeń, który należy pobrać ze strony:
https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec Są to zagnieżdzenia dla języka polskiego uzyskane systemem fastText.

Do przestrzeni wczytujemy tylko 100 tys. najczęstrzych słów, tak aby operacje przebiegały szybciej.

In [114]:
import io
def load_vectors(fname, limit = 100000):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    n = min(n,limit)
    embeddings = np.empty((n,d), dtype = np.float)
    words_idx = []
    for i, line in enumerate(fin):
        if i >= limit:
            break
        tokens = line.rstrip().split(' ')
        words_idx.append(tokens[0])
        embeddings[i] =  np.array(tokens[1:]).astype(np.float)
    return words_idx, embeddings
words_idx, embeddings = load_vectors('wiki.pl.vec')

Poniższe zadania mają na celu poekserymentowanie z przestrzenią zagnieżdżeń, ale też zrozumienie stojącymi za nich operacji. Dozwolone jest korzystanie tylko z podstawowych operatorów Python i numpy (w szczególności zakaz dotyczy: sklearn, gensim, fasttext itd.)

Jeśli potrzebujesz do dalszego przetwarzania przeprowadzenia jakichś normalizacji macierzy -- możesz wstępnie przetworzyć macierz zagnieżdzeń poniżej. Pamiętaj, że sprawdzarka będzie używała wywołań do `embeddings` (i `words_idx`) -- musisz nadpisać macierz zagnieżdzeń. To pole jest pomocnicze i nie podlega ocenie.

In [115]:
if not isinstance(words_idx, dict):
    words_idx = { word: i for i, word in enumerate(words_idx) }

Zaimplementuj funkcję, która obliczy podobieństwo kosinusowe pomiędzy dwoma wyrazami.

In [121]:
def calc_vector_sim(vector_a, vector_b):
    dot_product = vector_a.dot(vector_b)
    norm_a = vector_a.dot(vector_a) ** 0.5
    norm_b = vector_b.dot(vector_b) ** 0.5
    return dot_product / (norm_a * norm_b)

def calc_sim(word_a, word_b, words_idx, embeddings):
    vector_a = embeddings[words_idx[word_a], :]
    vector_b = embeddings[words_idx[word_b], :]
    return calc_vector_sim(vector_a, vector_b)

In [122]:
from nose.tools import assert_almost_equal
assert_almost_equal(calc_sim("bieber", "rihanna", words_idx, embeddings), calc_sim("rihanna", "bieber", words_idx, embeddings))

Policz podobieństwo pomiędzy wyrazem `bieber` a wyrazami:
    - `rihanna`
    - `piłsudski`
    - `kanada`
    - `polska`
    - `piosenka`

In [123]:
calc_sim("bieber", "rihanna", words_idx, embeddings)

0.524583248263655

In [124]:
for word in ['piłsudski', 'kanada', 'polska', 'piosenka']:
    print(word, calc_sim('bieber', word, words_idx, embeddings))

piłsudski 0.1930395805146356
kanada 0.20042742126487934
polska 0.12505934735679372
piosenka 0.2874871368858332


Zaimplementuj funkcję, która zwróci najbardziej podobne słowa (miara kosinusowa) do danego słowa `word`. W wyniku wypisz tylko `top` słów z najbliższymi zagnieżdżeniami, pomijając słowo `word`.

In [125]:
import heapq

def find_similar_to_vector(vector, words_idx, embedding_matrix, top=10):
    top_words = []
    for word, idx in words_idx.items():
        word_vector = embedding_matrix[idx]
        similarity = calc_vector_sim(vector, word_vector)
        item = (similarity, word)
        if len(top_words) <= top:
            heapq.heappush(top_words, item)
        else:
            heapq.heapreplace(top_words, item)
    return [item[1] for item in heapq.nlargest(top, top_words)]

def find_similar(word, words_idx, embedding_matrix, top=10):
    vector = embedding_matrix[words_idx[word]]
    similar = find_similar_to_vector(vector, words_idx, embedding_matrix, top + 1)
    if word in similar:
        similar.remove(word)
    return similar

In [126]:
assert len(find_similar("radość", words_idx, embeddings)) == 10

Znajdź najbardziej podobne słowa do kobieta, politechnika, mateusz, szczecin, niemcy, piłsudski

In [127]:
find_similar("kobieta", words_idx, embeddings)


['kobietą',
 'dziewczyna',
 'mężczyzna',
 'kobietę',
 'dziewczynka',
 'mężczyznę',
 'staruszka',
 'mężczyzną',
 'kobiecie',
 'mężczyzny']

In [128]:
for word in ['politechnika', 'mateusz', 'szczecin', 'niemcy', 'piłsudski']:
    print(word)
    print(find_similar(word, words_idx, embeddings))
    print()

politechnika
['politechniki', 'politechniką', 'politechnikę', 'politechniczny', 'politechnice', 'politechnicznej', 'politechnicznego', 'politechnicznym', 'inżynierska', 'elektrotechnika']

mateusz
['łukasz', 'bartłomiej', 'bartosz', 'kacper', 'marcin', 'mateusza', 'tomasz', 'patryk', 'rafał', 'mateuszem']

szczecin
['szczecinek', 'szczeciński', 'szczecinem', 'gryfino', 'szczecinie', 'stargard', 'szczecina', 'koszalin', 'szczecińska', 'świnoujście']

niemcy
['niemieccy', 'naziści', 'alianci', 'okupanci', 'polacy', 'hitlerowcy', 'niemieckie', 'rosjanie', 'niemców', 'niemcom']

piłsudski
['piłsudskim', 'piłsudskiego', 'piłsudskiemu', 'sosnkowski', 'mościcki', 'śmigły', 'józef', 'żeligowski', 'piłsudczyków', 'sosnkowskiego']



Krótko skomentuj wyniki dla słowa `niemcy`. Które z powstałych analogii biorą się z semantycznego powiązania a które z semantycznego podobieństwa?

In [130]:
# wiele powiązań bierze się z historii II wojny światowej
{
    'powiązane': [ # występują obok siebie
        'naziści', 'alianci', 'okupanci', 'polacy', 'hitlerowcy', 'rosjanie'
    ],
    'podobne': [ # występują na podobnych pozycjach w zdaniu
        'niemieccy', 'niemieckie',  'niemców', 'niemcom'
    ]
}

Zaimplementuj funkcje szukającą brakującego elementu relacji ,,`word_a` jest do `word_a2` jak `word_b` jest do...''. Funkcja powinna zwrócić 10 najbardziej pasujących słow z pominięciem słów będących jej argumentami.

In [131]:
def find_similar_pair(word_a, word_a2, word_b,  words_idx, matrix, top=10):
    vector_a = matrix[words_idx[word_a], :]
    vector_a2 = matrix[words_idx[word_a2], :]
    vector_b = matrix[words_idx[word_b], :]
    ideal_vector_b2 = vector_b - vector_a + vector_a2
    matches = find_similar_to_vector(ideal_vector_b2, words_idx, matrix, top + 3)
    excluded = {word_a, word_a2, word_b}
    return [word for i, word in enumerate(matches) if word not in excluded and i <= top]

In [132]:
assert find_similar_pair( "mężczyzna", "król", "kobieta", words_idx, embeddings)[0] == "królowa"

Pieniądze są do profesora jak wiedza do...

In [133]:
find_similar_pair("pieniądze", "profesor", "wiedza", words_idx, embeddings)

['habilitowany',
 'docent',
 'wykładowca',
 'profesorem',
 'habilitacja',
 'adiunkt',
 'rektor',
 'profesora',
 'humanistycznych']

Mateusza jest do mateusz jak łukasza do ...

In [134]:
find_similar_pair("mateusza", "mateusz", "łukasza", words_idx, embeddings)

['łukasz',
 'bartłomiej',
 'bartosz',
 'maciej',
 'tomasz',
 'rafał',
 'patryk',
 'marcin',
 'michał']

Warszawa jest do "polska" jak "berlin" do ...

In [135]:
find_similar_pair("warszawa", "polska", "berlin", words_idx, embeddings)

['niemiecka',
 'berliner',
 'wschodnioniemiecka',
 'berlińska',
 'deutschland',
 'deutsche',
 'brandenburg',
 'berlinem',
 'germany']

Zurich jest do ETH jak Poznań do ...

In [136]:
find_similar_pair("zurich", "eth", "poznań", words_idx, embeddings)

['„poznań',
 'wrocław',
 'poznania',
 'poznańskie',
 'uam',
 'poznaniu',
 'kraków',
 'gniezno',
 'poznańską']

Niemcy są do Merkel jak Polska do ...

In [137]:
find_similar_pair("niemcy", "merkel", "polska", words_idx, embeddings)

['kaczyńska',
 'lewandowska',
 'kwaśniewska',
 'ekonomistka',
 'lekarka',
 'parlamentarzystka',
 'marcinkiewicz',
 'olszewska',
 'bezpartyjna']

Na wektorach możemy wykonywać standardowe operacje algebry liniowej takie jak np. projekcja czyli rzutowanie danych na jakichś zbiór osi (więcej: notatki z algebry liniowej np. https://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/least-squares-determinants-and-eigenvalues/projections-onto-subspaces/). W szczególności może to się przydać do zrzutowania słowa na przestrzeń w której pewny wybrany kierunek (wskazywany przez wektor) jest eliminowany.

Do czego może to się przydać? Jeśli uruchomisz funkcję `find_similar` dla słowa ,,mateusza'' znajdziesz m.in. ,,łukasza'' ale także ,,ewangelia'', ,,ewangelisty'' i ,,apostoła''. Chcąc pominąc kontekst religijny tego słowa możesz zrzutować jego reprezentacje na przestrzeń bez wektora ,,ewangelia'' i poszukać jego najbliższych sąsiadów (którymi będą teraz po prostu imiona męskie). Zaimplementuj taką funkcję.


In [149]:
def find_similar_with_rejection(word, remove, words_idx, matrix, top=10):
    """
    Działanie analogiczne do find_similar z dodatkowym parametrem remove, 
    który jest *listą* słów, które należy wyrzucić poprzez projekcję.
    Dla remove=[] powinno się zwracać dokładnie to samo co find_similar
    """
    projection = np.eye(matrix.shape[1])
    for removed_word in remove:
        direction = matrix[words_idx[removed_word], :]
        next_projection = np.outer(direction, direction) / np.inner(direction, direction)
        projection = projection.dot(next_projection)
    projected_matrix = matrix.dot(projection)
    return find_similar(word, words_idx, projected_matrix, top)
print ("Standardowe poszukiwanie:", find_similar_with_rejection("mateusza",[] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings))

Standardowe poszukiwanie: ['łukasza', 'ewangelii', 'ewangelisty', 'ewangelia', 'bartłomieja', 'ewangeliach', 'apostoła', 'mateusz', 'tymoteusza', 'jakuba']
Poszukiwanie po projekcji: ['zwróci', 'zgromadzić', 'yagnik', 'translated', 'stanhope', 'skokach', 'robotników', 'releasing', 'proponuje', 'prezentowały', 'pisarstwa']


In [140]:
assert "ewangelii" in find_similar_with_rejection("mateusza",[] , words_idx, embeddings)
assert "ewangelii" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)
assert "ewangelisty" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)


Analogicznie słowo ,,java'' jest nie tylko nazwą języka programownia (https://pl.wikipedia.org/wiki/Java_(ujednoznacznienie)) -- jest np. nazwą geograficzną (indonezyjska wyspa koło Sumatry). Sprawdź jakie wyrazy są podobne do "java" oraz po odrzuceniu kierunku "javascript" (tj. kierunku związanego z językami programowania).

In [141]:
find_similar_with_rejection("java", ["javascript"] , words_idx, embeddings)

['żołd',
 'znanej',
 'wyspowy',
 'wulkanicznego',
 'written',
 'ubrań',
 'strefie',
 'serena',
 'scania',
 'rzeczpospolitą',
 'rossetti']

Spróbuj poekseprymentować samemu!

In [147]:
find_similar_with_rejection("inteligencja", ['sztuczna'] , words_idx, embeddings)

['ziemnej',
 'zadná',
 'oberst',
 'niewielki',
 'garlicki',
 'etylenu',
 'charakterystyczną',
 '˚',
 'żywcem',
 'życiorysy',
 'żeliwnych']

Wykonanie projekcji w przestrzeni zagnieżdżeń może być jedną z prostych technik zwalczenia tzw. gender bias (http://wordbias.umiacs.umd.edu/) w reprezentacji słów. Okazuje się, że wykonanie projekcji macierzy zagnieżdżeń na przestrzeń w której ,,brakuje kierunku he-she'' może być bardzo prostą techniką zredukowania tego typu obciążenia.

## Zadanie 2 - zagnieżdżenia dokumentów
W tym ćwiczeniu powócimy do zbioru tweetów, który analizowaliśmy w poprzednim dokumencie.

In [19]:
from helpers import DataSet
training_set = DataSet(['tweets.txt'])

Reading data set ['tweets.txt']


In [20]:
for i in training_set.tweets:
    print(i.text)
    print(i.tokens)
    print(i.clazz)
    break

dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon.
['dear', '@microsoft', 'the', 'newooffice', 'for', 'mac', 'is', 'great', 'and', 'all', ',', 'but', 'no', 'lync', 'update', '?', "c'mon", '.']
negative


Tym razem do zbudowania reprezentacji będziemy używać narzędzie Universal Sentence Encoder stworzone przez Googla na bazie głębokiej sieci uśredniającej (i architektur rekurencyjnych). Poniższy kod pokazuje sposób użycia tego narzędzia. 
Kod spokojnie można wywoływać na CPU -- choć ściąganie modelu trochę może potrwać.

In [21]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])
embeddings

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.03133018, -0.06338634, -0.01607502, ..., -0.03242778,
        -0.04575741,  0.05370456],
       [ 0.05080861, -0.01652428,  0.01573781, ...,  0.00976659,
         0.03170123,  0.0178812 ]], dtype=float32)>

Wykorzystując reprezetnację USE wytrenuj wybrany klasyfikator z pakietu `sklearn` i zweryfikuj jego jakość działania.

In [31]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

size = len(training_set.tweets)
train_size = int(size * 0.75)
train = training_set.tweets[0:train_size]
test = training_set.tweets[train_size+1:]

train_sentences = embed([tweet.text for tweet in train])
train_classes = [tweet.clazz for tweet in train]

test_sentences = embed([tweet.text for tweet in test])
test_classes = [tweet.clazz for tweet in test]

classifier = SVC()
classifier.fit(train_sentences, train_classes)
predicted = classifier.predict(test_sentences)
print(accuracy_score(test_classes, predicted))

0.6539589442815249

Skomentuj wyniki i odnieś je do wyników z poprzedniego zadania domowego. Na ile użycie reprezentacji rozproszonych pozwoliło na poprawę wyników?

In [34]:
"""
Trafność uzyskana tą metodą bez żadnych dodatkowych ulepszeń
jest o ok. 7 punktów procentowych wyższa od podejścia
z haszowaniem cech lub grupowaniem.
"""

# Zadanie 3 - konstruowanie zagnieżdżeń
W tym ćwiczeniu kontynuujemy pracę z tweetami, ale pomijamy całkowicie ich klasy. Zbiór tweetów potraktujemy jako korpus do nauczenia zagnieżdżeń słów przy pomocy macierzy PMI.
- Wypełnij macierz kontekst - dokument przy użyciu symetrycznego okna o promieniu 4 (po 4 słowa w każdą stronę)
- Możesz ograniczyć słownictwo do 10K słów
- Przekształć macierz w macierz PPMI
- Stwórz zagnieżdżenia wykorzystując dekompozycję SVD do wybranej wymiarowości $d$ (ze względu na koszt obliczeniowy może to być mała wymiarowość np. $d=10$)

In [104]:
from collections import Counter
from sklearn.decomposition import TruncatedSVD
from helpers import RE_HTTP, RE_HASHTAG, RE_EMOTICONS

reject_patterns = [RE_HTTP, RE_HASHTAG, RE_EMOTICONS]

def filter_tokens(tokens):
    return [
        token for token in tokens 
        if not any(pattern.match(token) for pattern in reject_patterns)
    ]

def build_vocabulary(corpus, size):
    word_counter = Counter()
    for tweet in corpus:
        filtered_tokens = filter_tokens(tweet.tokens)
        word_counter.update(filtered_tokens)

    return { word: index for index, (word, _) in enumerate(word_counter.most_common(size)) }

def slide_window(tokens, radius):
    length = len(tokens)
    for offset in range(-radius, length - radius):
        start_pos = max(offset, 0)
        central_pos = offset + radius
        end_pos = max(offset + 2 * radius + 1, 0)
        
        left_context = tokens[start_pos:central_pos]
        word = tokens[central_pos]*
        right_context = tokens[central_pos+1:end_pos]
        yield left_context, word, right_context

def build_word_context_matrix(corpus, vocabulary, radius):
    word_context = np.zeros((len(vocabulary), len(vocabulary)))
    for tweet in corpus:
        for left, word, right in slide_window(tweet.tokens, radius):
            if word not in vocabulary:
                continue
            context = left + right
            for other_word in context:
                if other_word not in vocabulary:
                    continue
                word_context[vocabulary[word], vocabulary[other_word]] += 1
    return word_context

def calculate_ppmi(context_document):
    total = context_document.sum()
    row_margin = context_document.sum(axis=0, keepdims=True) / total
    col_margin = context_document.sum(axis=1, keepdims=True) / total
    association = (context_document / total) / col_margin.dot(row_margin)
    association[association == 0] = 1
    return np.log2(association)

def reduce_dimensionality(matrix, dim):
    svd = TruncatedSVD(dim)
    return svd.fit_transform(matrix)

def train_embeddings(corpus, embedding_dim, radius, vocab_size):
    print('building vocabulary...')
    vocabulary = build_vocabulary(corpus, vocab_size)
    print('building word-context matrix...')
    word_context = build_word_context_matrix(corpus, vocabulary, radius)
    print('calculating PPMI...')
    ppmi = calculate_ppmi(word_context)
    print('applying SVD...')
    embeddings = reduce_dimensionality(ppmi, embedding_dim)
    print('done!')
    return embeddings, vocabulary

embeddings, vocabulary = train_embeddings(
    corpus = training_set.tweets,
    embedding_dim = 10,
    radius = 4,
    vocab_size = 10000,
)

building vocabulary...
building word-context matrix...
calculating PPMI...
applying SVD...
done!


Przetestuj działanie Twoich zagnieżdżeń wykorzystując funkcję `find_similar` na wybranych słowach.

In [111]:
print(find_similar('microsoft', vocabulary, embeddings))

['munchkin', 'bafana', 'revise', 'celeb.tc', 'unlucky', '11yo', 'omlis', 'mynextbigproject', 'homer', 'inventory', 'o.o']
